In [43]:
import pandas as pd

# symbols = {'all': ["EURCHF", "EURNZD", "NZDUSD", "AUDNZD", "USDJPY", "NZDJPY", "GBPJPY", "USDCHF"]}
symbols = {'all': ["USDCHF"]}
"""Đoạn code này có thể chỉnh sửa lấy loại tiền tệ theo yêu cầu"""
def get_data(symbols):
  data_forex = {}
  for sym in symbols["all"]:
    data_forex[sym] = pd.read_csv(f'D:/ForexResearch/Data/{sym}.csv', index_col=0)
    print(sym)
  return data_forex

data_forex = get_data(symbols)

USDCHF


# Bước lưu lại dữ liệu và load dữ liệu lên nếu cần

In [44]:
import os
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import pickle

from datetime import datetime
from collections import Counter
from finta import TA
from scipy.signal import savgol_filter
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tqdm import tqdm
import os
from pathlib import Path
import glob
# thư viện làm trơn nhãn
import simdkalman
from sklearn.ensemble import IsolationForest
import plotly.express as px
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from tqdm import notebook
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")


# Công đoạn này thêm các technical indicators vào

In [45]:
# Hàm khởi tạo các  technical indicator
def create_indicators(ohlcv: pd.DataFrame) -> pd.DataFrame:
    data = ohlcv.copy()
    ohlcv = ohlcv.apply(pd.to_numeric, errors='coerce')
    indi, signal = bias(ohlcv)
    data = pd.concat([data, indi], axis=1)
    data['BIAS_signal'] = signal
    data['VR'], data["VR_signal"] = vr(ohlcv)
    data['TRIX'], data["TRIX_signal"] = trix(ohlcv)
    data['ER'] = TA.ER(ohlcv)
    data['EVWMA'] = TA.EVWMA(ohlcv)
    data['VWAP'] = TA.VWAP(ohlcv)
    data['MOM'] = TA.MOM(ohlcv)
    data['ROC'] = TA.ROC(ohlcv)
    data['RSI'] = TA.RSI(ohlcv)
    data['IFT_RSI'] = TA.IFT_RSI(ohlcv)
    data['ATR'] = TA.ATR(ohlcv)
    data['BBWIDTH'] = TA.BBWIDTH(ohlcv)
    data['ADX'] = TA.ADX(ohlcv)
    data['STOCH'] = TA.STOCH(ohlcv)
    data['STOCHD'] = TA.STOCHD(ohlcv)
    data['AO'] = TA.AO(ohlcv)
    data['MI'] = TA.MI(ohlcv)
    data['MFI'] = TA.MFI(ohlcv)
    data['PZO'] = TA.PZO(ohlcv)
    data['EFI'] = TA.EFI(ohlcv)
    data['EMV'] = TA.EMV(ohlcv)
    data['CCI'] = TA.CCI(ohlcv)
    data['FISH'] = TA.FISH(ohlcv)
    data['FVE'] = TA.FVE(ohlcv)

    macd = TA.MACD(ohlcv)
    data['MACDCal'] = macd['MACD'] - macd['SIGNAL']

    macdev = TA.EV_MACD(ohlcv)
    data['EVMACD'] = macdev["MACD"]

    data['TR'] = TA.TR(ohlcv)

    DMI = TA.DMI(ohlcv)
    data['DMI+'] = DMI["DI+"]
    data['DMI-'] = DMI["DI-"]

    # VORTEX = TA.VORTEX(ohlcv)
    # data['VIp'] = VORTEX["VIp"]
    # data['ADL'] = TA.ADL(data)

    TSI = TA.TSI(ohlcv)
    data['TSI'] = TSI["TSI"]
    data['TSIsignal'] = TSI["signal"]

    KST = TA.KST(ohlcv)
    data['KST'] = KST["KST"]

    data['CHAIKIN'] = TA.CHAIKIN(ohlcv)
    data['OBV'] = TA.OBV(ohlcv)
    data['WOBV'] = TA.WOBV(ohlcv)

    EBBP = TA.EBBP(ohlcv)
    data['EBBPBull'] = EBBP["Bull."]
    data['EBBPBear'] = EBBP["Bear."]

    BASPN = TA.BASPN(ohlcv)
    data['BASPNBuy'] = BASPN["Buy."]
    data['BASPNSell'] = BASPN["Sell."]
    data['COPP'] = TA.COPP(ohlcv)

    BASP = TA.BASP(ohlcv)
    data['BASPBuy'] = BASP["Buy."]
    data['BASPSell'] = BASP["Sell."]

    WTO = TA.WTO(ohlcv)
    data['WTOWT1'] = WTO["WT1."]

    data['STC'] = TA.STC(ohlcv)
    data['VPT'] = TA.VPT(ohlcv)

    # Có 50 kỹ thuật technical indicators ở đây
    return data

LOOK_BACK = 10
"""Hàm này dùng để phân chia data ra theo từng ngày nối đuôi nhau vựa trên lock_back"""
# Xem xem trung bình 10 ngày tăng hay giảm, là dùng để xem xu hướng của tiền

def create_dataset(data, label, look_back=1):  #
    X_ = []
    y_ = []
    # Tạo vòng lạp để lấy các lookback
    for i in range(len(data)-look_back-1):
        X_.append(data[i:(i+look_back)])
        y_.append(label[i + look_back])
    return np.array(X_), np.array(y_)

""" Hàm này để lấy các ngày liên tiếp theo n_days để làm các cột dữ liệu"""

def add_past_days_as_feature(data: pd.DataFrame, n_days: int = 5):
    data = pd.concat([data.shift(i).add_suffix(f"_{i}") for i in range(n_days)], axis=1)
    return data

" Hàm Bias dùng để so sánh giữ hai hai tính hiệu dài hạn và ngắn hạn vựa trên short_val và long_val "
def bias(prices):
    short_avg = prices['Close'].rolling(3, min_periods=1).mean()
    long_avg = prices['Close'].rolling(5, min_periods=1).mean()

    short_val = pd.Series(((prices['Close'] - short_avg) / short_avg) * 100, name="BIAS_short", index=prices.index)
    long_val = pd.Series(((prices['Close'] - long_avg) / long_avg) * 100, name="BIAS_long", index=prices.index)
    indi = pd.concat([short_val, long_val], axis=1)

    # So sánh xem dài hạn hay ngắn hạn cái nào sẽ lời hơn
    signal = pd.Series((long_val > short_val).astype(int), name="BIAS_signal", index=prices.index)
    return indi, signal

def vr(prices):
    maximum = (prices['High'] + prices['Close'].shift(1).bfill()).mean() # Lấy giá cao nhất ngày hiện tạo cộng cho ngày đóng cửa của tương lai,
    # Nếu tương lai là Nan thì cộng cho giá đóng cưa hiện tại
    minimum = (prices['Low'] + prices['Close'].shift(1).bfill()).mean()
    high = prices['High'].rolling(14, min_periods=1).mean()
    low = prices['Low'].rolling(14, min_periods=1).mean()

    # Tính chỉ số
    indi = pd.Series((maximum - minimum) / (high - low), name="VR", index=prices.index)
    signal = pd.Series((indi > 0.5).astype(int), name="VR_signal", index=prices.index)
    return indi, signal

# Hàm tính giá trị Trix
def trix(prices):
    indi = TA.TRIX(prices, 10)
    signal = pd.Series((indi < 0).astype(int), name="TRIX_signal", index=prices.index)
    return indi, signal

In [46]:

# hàm này để đánh giá mô hình tổng
# Là khi chúng ta train ra hết 3 model chúng ta xem rằng chúng kết hợp với nhau thì kết quả cuối cùng độ chính xác là bao nhiêu
def CV_ensemble(ensemble_name, ensemble_func, estimators, X_train, y_train, n_folds=5, shuffle=True, random_state=2022):
  kf = KFold(n_splits=5, random_state=random_state, shuffle=True) # Tạo số lượng dữ liệu bằng Kfold sau đó shuffle

  res_list = [] # tạo res_list lưu thông tin của từng lần chia kfold
  # Chạy tập dữ liệu Kfold
  for train_idx, valid_idx in notebook.tqdm(kf.split(X_train), total=kf.get_n_splits(), desc='Eval_CV'): # Chạy theo số lần n_splits
    X_train_train, X_valid = X_train[train_idx], X_train[valid_idx]
    y_train_train, y_valid = y_train[train_idx], y_train[valid_idx]
    # Tổng hợp kết quả test đối vơi từng cặp k riêng
    ensemble_pred_proba = ensemble_func(estimators, X_train_train, y_train_train, X_valid)


    # Lấy các đánh giá  của model với tập train test k này
    neg_log_loss = np.negative(log_loss(y_valid, ensemble_pred_proba))
    accuracy = accuracy_score(y_valid, ensemble_pred_proba.argmax(axis=1))

    res_list.append([ensemble_name, neg_log_loss, accuracy]) # Lưu kết quả lại`
  res_df = pd.DataFrame(np.vstack((res_list)))
  res_df.columns = ['model', 'log_loss', 'accuracy']  # thêm các thông số để đánh giá
  return res_df.reset_index(drop=True)


# hàm này để dự đoán bằng cách tổng hợp 3 model có train lại
def ensemble_average(estimators, X_train, y_train, X_test):
  # Hàm này vựa trên việc lấy xác suất của các model vote cái nào có xác suất cao nhất thì chọn
  preds = []
  num_estimators = len(estimators)
  num_class = len(np.unique(y_train))
  for iter in range(num_estimators):
    y_train = np.array(y_train, dtype=np.int64)
    estimators[iter].fit(X_train, y_train)
    preds.append(estimators[iter].predict_proba(X_test))


  preds_stack = np.hstack((preds))
  preds_mean = []
  for iter in range(num_class):
    col_idx = np.arange(iter, num_estimators * num_class, num_class)
    preds_mean.append(np.mean(preds_stack[:,col_idx], axis=1))

  avg_pred = np.vstack((preds_mean)).transpose()
  return avg_pred


# hàm này đưa ra dự đoán mà ko cần train lại
def ensemble_average_model(estimators, X_test, y_train): # esitamtors là các model và X_test là dữ liệu train
  preds = []
  num_estimators = len(estimators)
  num_class = len(np.unique(y_train))
  # đoạn này lấy tổng dữ đoán của các model trong estimators
  for iter in range(num_estimators):

    preds.append(estimators[iter].predict_proba(X_test))

  preds_stack = np.hstack((preds))
  preds_mean = []
  # Dự đoán trung bình cho mỗi nhãn
  for iter in range(num_class):
    col_idx = np.arange(iter, num_estimators * num_class, num_class) # Tính cho số cột cho nhãn được dự đoán
    preds_mean.append(np.mean(preds_stack[:,col_idx], axis=1)) # Tính giá trị trung bình cho dự đoán

  avg_pred = np.vstack((preds_mean)).transpose() # Lấy hết các giá trị tring bình sau đó chuyển vị
  return avg_pred
# hàm tạo ngưỡng nhiễu
def outlier_threshold(normality, k=1.5):
  q1 = np.quantile(normality, 0.2)
  q3 = np.quantile(normality, 0.8)
  threshold = q1 - k*(q3-q1) # Công thưc tạo ngưỡng nhiễu
  return threshold

# Khởi tạo và xử lý dữ liệu

In [47]:

def create_data(symbols, data_forex):
  price = {}
  # Lấy data của giá đóng cửa của loại đồng tiền
  for symbol in data_forex.keys():
      price[symbol] = data_forex[symbol]['Close'].copy()
  y = {}
  for symbol in data_forex.keys():
      y[symbol] = data_forex[symbol]["Close"].ffill()
  # x là features
  X = {}
  # Đoạn này lấy dữ liệu X là data dùng để tách train, test
  for symbol in data_forex.keys():
      X_base_features = create_indicators(data_forex[symbol])
      X[symbol] = add_past_days_as_feature(data=X_base_features, n_days=LOOK_BACK)
      X[symbol]["label"] = y[symbol].copy()
      X[symbol] = X[symbol].dropna(axis=0)
      y[symbol] = X[symbol]["label"].copy()
  return X, y
X, y  = create_data(symbols, data_forex)
    # print(symbol, X[symbol].shape)

In [48]:
from sklearn.model_selection import train_test_split
config = {
    'data_name': '3d_forex',
    'random_state': 2023
}
# Hàm này tách dữ liệu và sau đó chuẩn hóa riêng cho từng mã
def split_data_scale_2(X: dict, y: dict, symbols: dict):
    data_all_money = {}
    for symbol in symbols["all"]:
        # Lấy dữ liệu gốc khi (chưa scale)
        data_original = X[symbol].copy()
        y_ = pd.Series(y[symbol])
        # Khởi tạo biến để lưu dữ liệu train và test cho từng fold của từng loại tề
        data_train_test = []

        # Chia dữ liệu thành tập huấn luyện và tập kiểm tra theo tỷ lệ 90/10
        X_train, X_test, y_train, y_test = train_test_split(data_original, y_, test_size=0.1,shuffle =False, random_state=config['random_state'])
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.fit_transform(X_test)

        data_all_money[symbol] = [X_train_scaled, X_test_scaled, y_train, y_test]
    return data_all_money # Dữ liệu được lưu ở dạng dict và 1 loại đồng tiền chưa 4 dữ liệu train và test của X và y

# data_all_money = split_data_scale_2(X, y, symbols) # Lấy dữ liệu đã scale ra



#  Ta đi smooth data để smooth ta cần biết các dữ liệu nhiễu thường xuyên, và bộ dữ liệu thực tế
# Việc cần ở đây là chỉ số nhiễu tượng trưng cho dữ liệu đó chỉ cần chỉnh bao nhiêu phần trăm bị nhiễu
def Smoothing_data(y, symbols) -> dict:
  symbolsmooths = {}
  # cho nhiều bộ smooths y khác nhau để xem cái nào tốt nhất
  for syms in symbols["all"]:
    smooths = []
    for isf in range(4,5):
        for ins in range(4,5):
            smoothing_factor = isf
            n_seasons = ins


            # --- define state transition matrix A tạo ma trận n_season+1
            state_transition = np.zeros((n_seasons+1, n_seasons+1))
            # hidden level         Đặt mặc định số đầu tiên là 1
            state_transition[0,0] = 1
            # season cycle         Sau đó ta chỉnh các thông sô khác cho mặc định dòng 1 có n_season = -1
            state_transition[1,1:-1] = [-1.0] * (n_seasons-1)

            # Tạo đường chéo chính có giá trị là 1
            state_transition[2:,1:-1] = np.eye(n_seasons-1)

            # --- observation model H
            # observation is hidden level + weekly seasonal compoenent
            observation_model = [[1,1] + [0]*(n_seasons-1)]

            # --- noise models, parametrized by the smoothing factor
            level_noise = 0.2 / smoothing_factor
            observation_noise = 0.2
            season_noise = 1e-3

            process_noise_cov = np.diag([level_noise, season_noise] + [0]*(n_seasons-1))**2
            observation_noise_cov = observation_noise**2
            # Sử dụng Kalman filter
            kf = simdkalman.KalmanFilter(
                state_transition, # Hiển thị cách model chuyển tiếp để thay đổi ( Đây là cách nhìn dữ liệu bị nhiễu)
                process_noise_cov, # ma trận biểu thị mức độ nhiễu của mô hình (Đây là để biết nhiễu thường xảy ra với tuần suât nào)
                observation_model, #  Là mô hình quan sát của hệ thông thường là ma trận hay hàm số biểu thị (Đây là  kết quả hiện tại đang cần chỉnh nhiễu)
                observation_noise_cov #  Ma trận biểu thị mức độ nhiễu trong đo lường  (Đây giống như là sai số để xem kết quả dự đoán với thực tế thì sau bnhieu)
                )
            # Sau đó ta tính toán lấy ra được các dự đoán nhiễu và trã về dự đoán bị nhiễu được quy định bằng các tham số trên
            block = y[syms]
            n_train = block.shape[0]
            n_test = 60
            result = kf.compute(block, n_test)
            predictproba = result.smoothed.states.mean[:,0]
            y_label = []
            for ivalue in range(1,len(predictproba)):
                if(predictproba[ivalue] > predictproba[ivalue-1]):
                    y_label.append(1)
                else:
                    y_label.append(-1)

            smooths.append(y_label)
    symbolsmooths[syms] = smooths
  return symbolsmooths

# symbolsmooths_y = Smoothing_data(y,symbols)


def smooth_label(y_train_ ,symbolsmooths, syms ):
  # Hàm chỉnh dữ liệu lại cho smooth
  y_train = []
  y_test = []
  # chạy từng mã gán lại nhãn đã smooth, -1 là 0 giảm, 1 là 1 tăng
  for idx in range(len(y_train_)):
      if symbolsmooths[syms][0][idx]  == -1:
          y_train.append(0)
      else:
          y_train.append(1)
  # y_test cũng vậy, phần test sẽ là phần còn lại tính từ n_split
  for idx in range(len(symbolsmooths[syms][0]) - len(y_train_)):
      if symbolsmooths[syms][0][idx+ len(y_train_)] == -1:
          y_test.append(0)
      else:
          y_test.append(1)
  y_test.append(1)
  return y_train, y_test


def create_train_test_all_data_forex(X, y, symbols):
  data_all = {}
  data_all_money = split_data_scale_2(X,y, symbols)
  symbolsmooth_y = Smoothing_data(y, symbols)

  for sym in symbols["all"]:
    # Lấy các cột của dữ liệu
    df = pd.DataFrame(X[sym] )
    col_df = df.columns

    # Lấy đúng loại tiền cần dùng
    X_train, X_test, y_train, y_test = data_all_money[sym]
    # Sau khi smooth data ta có y_train và y_test mới
    y_train, y_test = smooth_label( y_train, symbolsmooth_y, sym)

    # Sau đó  tạo ra các dataframe mới
    X_train = pd.DataFrame(X_train, columns = col_df)
    X_test =  pd.DataFrame(X_test, columns = col_df)
    # Drop các cột không cần thiết là label đi
    X_train = X_train.drop(columns = ["label"])
    X_test = X_test.drop(columns = ["label"])
    # Lấy dữ liệu tiền tệ đó ra
    data_set = pd.get_dummies(X_train, drop_first=False)
    data_all[sym] = [data_set, X_test, y_train, y_test]
  return data_all

data_all = create_train_test_all_data_forex(X, y, symbols)


In [49]:
# data_all

# Tuning hyperparameter cho các model và loại bỏ outlier trong dữ liệu:
1. Random forest
2. XGBoost
3. MLP

In [50]:
def create_clf(X_train):
  clf = IsolationForest(
    n_estimators=100,
    max_samples='auto',
    n_jobs=-1,
    random_state=config['random_state'])

  clf.fit(X_train)
  return clf


# Hàm loại bỏ outlier cho tập data train
def delete_outlier(clf, X_train, y_train):
  # Lấy ngưỡng cho dữ liệu
  normality_df = pd.DataFrame(clf.decision_function(X_train), columns=['normality'])
  threshold = outlier_threshold(normality_df['normality'].values, k=1.5)

  # Sau đo ta loại bỏ dòng bị outlier
  X_train = X_train[normality_df['normality'].values>=threshold]
  y_train = y_train[normality_df['normality'].values>=threshold]
  return X_train, y_train

# hàm này tạo ra dataframe chưa bộ thông số tốt nhất cho từng model
def create_best_cv(model_name):

  model_list = []
  # mỗi model chuẩn bị 5 cái loss tốt nhất vì chạy CV
  for name in model_name:
    model_list.append(np.full(5, name))

  best_cv_df = pd.DataFrame({'model': np.hstack((model_list)), 'accuracy':None, 'best_hyper_param':None})
  return best_cv_df

def data_np(data_all, symbols):
  for sym in symbols["all"]:
    # Thay đổi dữ liệu về dạng làm tròn và numpy

    data_all[sym][2] = np.array(data_all[sym][2])
    data_all[sym][3] = np.array(data_all[sym][3])

    data_all[sym][0] = data_all[sym][0].astype(np.float32)
    data_all[sym][1] = data_all[sym][1].astype(np.float32)

def delete_and_create_best(data_all, symbols):
  data_np(data_all, symbols)
  best_cv_df = {}
  for sym in symbols["all"]:
    # Tạo clf cho delete outlier
    clf = create_clf(data_all[sym][0])
    # xóa các outliers
    print(sym)
    data_all[sym][0], data_all[sym][2] = delete_outlier(clf, data_all[sym][0], data_all[sym][2] )
    # Tạo ra best cv cho từng mã tiền
    model_name = [ 'rf', 'xgb', 'mlp']
    best_cv_df[sym] = create_best_cv(model_name)
  return best_cv_df



In [51]:
best_cv_df = delete_and_create_best(data_all, symbols)

USDCHF


In [52]:
from sklearn.model_selection import GridSearchCV
def optimizer_rf_model(X_train, y_train, best_cv_df, sym):
  #from keras.callbacks import EarlyStopping
  param_grid = {
    'n_estimators': [200,300,400],
    'max_depth' : [8,10],
    'criterion' :['entropy']
  }
  rfc=RandomForestClassifier(random_state=23)
  tune_search = GridSearchCV(estimator=rfc, param_grid=param_grid,n_jobs =6,scoring='accuracy', verbose =2,refit='accuracy',cv= 3)
  tune_search.fit(X_train, y_train)
  model_name = 'rf'
  accuracies = tune_search.cv_results_["mean_test_score"]
  best_hyperparameter = tune_search.cv_results_["params"]
  # Lấy ra 5 parameter tốt nhất
  top_5_indices = (-accuracies).argsort()[:5]
  top_5_hyperparameter = [best_hyperparameter[i] for i in top_5_indices]
  cv_df = pd.DataFrame(tune_search.cv_results_)
  # Lấy trung bình dự đoán độ chinh xác khi trên tập kfold của tùng best hyperparameter
  acc = []
  for i in range(len(cv_df)):
    if cv_df["params"].values[i] in top_5_hyperparameter:
      a = [cv_df["split0_test_score"].values[i], cv_df["split1_test_score"].values[i], cv_df["split2_test_score"].values[i]]
      acc.append(round(sum(a)/3, 5)) # Tính trung bình cho từng hyperparameter khi làm xong việc test fold = 3
  best_cv_df[sym].loc[best_cv_df[sym]['model']==model_name, 'best_hyper_param'] = top_5_hyperparameter
  best_cv_df[sym].loc[best_cv_df[sym]['model']==model_name, 'accuracy'] = acc
  # lưu lại 5 model tốt nhất bằng random forest
  return tune_search

In [53]:
# XGB được tối ưu bằng Grid search
def optimizer_xgb_model(X_train, y_train, best_cv_df, sym):
  parameters = {
      # Các Hyper parameter cần tính toán
    'n_estimators': [100],
    'learning_rate': [0.1,0.05],
    'min_child_weight': [5],
    'gamma': [0.5],
    'subsample': [0.6],
    'colsample_bytree': [1.0, 2],
    'max_depth': [5,8],
    'objective': ['binary:logistic'],
    'use_label_encoder': [False],
    'random_state': [config['random_state']]
}
  xgb_model = XGBClassifier()

  grid_search = GridSearchCV(
      estimator=xgb_model,
      param_grid=parameters,
      scoring = 'accuracy',
      n_jobs = 10,
      cv = 3,
      verbose=2
  )

  grid_search.fit(X_train, y_train)
  model_name = 'xgb'
  accuracies = grid_search.cv_results_["mean_test_score"]
  best_hyperparameter = grid_search.cv_results_["params"]
  # Lấy ra 5 parameter tốt nhất
  top_5_indices = (-accuracies).argsort()[:5]
  top_5_hyperparameter = [best_hyperparameter[i] for i in top_5_indices]
  cv_df = pd.DataFrame(grid_search.cv_results_)
  # Lấy trung bình dự đoán độ chinh xác khi trên tập kfold của tùng best hyperparameter
  acc = []
  for i in range(len(cv_df)):
      if cv_df["params"].values[i] in top_5_hyperparameter:
        a = [cv_df["split0_test_score"].values[i], cv_df["split1_test_score"].values[i], cv_df["split2_test_score"].values[i]]
        acc.append(round(sum(a)/3, 5)) # Tính trung bình cho từng hyperparameter khi làm xong việc test fold = 3
  best_cv_df[sym].loc[best_cv_df[sym]['model']==model_name, 'best_hyper_param'] = top_5_hyperparameter
  best_cv_df[sym].loc[best_cv_df[sym]['model']==model_name, 'accuracy'] = acc
  return grid_search

In [54]:
def best_cv_rf_and_xgb(data_all, symbols, best_cv_df):
  tune_and_grid = {}
  for sym in symbols["all"]:
    X_train = data_all[sym][0]
    y_train = data_all[sym][2]
    # lấy các search của tất cả model
    tune_search = optimizer_rf_model(X_train, y_train, best_cv_df, sym)
    grid_search = optimizer_xgb_model(X_train, y_train, best_cv_df, sym)
    tune_and_grid[sym] = [tune_search, grid_search]
  return tune_and_grid

In [55]:
# Hàm tối ưu MLP bằng Grid_search
# Hàm dưới đây dung để Tuning hyperparameter cho model MLP
def optimize_model_MLP(X_train, y_train, best_cv_df, sym):
  mlp_clf = MLPClassifier()
  param_grid = {
      'hidden_layer_sizes': [(120,80,40),(100,60,30)],
      'max_iter': [80, 100],
      'activation': ['tanh'],
      'solver': ['sgd'],
      'alpha': [0.05,0.1],  # Chỉ số cần được chỉnh
      'learning_rate': ['constant','adaptive'],
  }
  # Đoạn dưới đây là chạy Grid search cho model
  grid_search = GridSearchCV(mlp_clf, param_grid, n_jobs= 6, cv=3)
  grid_search.fit(X_train, y_train)
  model_name = 'mlp'
  accuracies = grid_search.cv_results_["mean_test_score"]
  best_hyperparameter = grid_search.cv_results_["params"]
  # Lấy ra 5 parameter tốt nhất
  top_5_indices = (-accuracies).argsort()[:5]
  top_5_hyperparameter = [best_hyperparameter[i] for i in top_5_indices]
  cv_df = pd.DataFrame(grid_search.cv_results_)
  # Lấy trung bình dự đoán độ chinh xác khi trên tập kfold của tùng best hyperparameter
  acc = []
  for i in range(len(cv_df)):
    if cv_df["params"].values[i] in top_5_hyperparameter:
      a = [cv_df["split0_test_score"].values[i], cv_df["split1_test_score"].values[i], cv_df["split2_test_score"].values[i]]
      acc.append(round(sum(a)/3, 5)) # Tính trung bình cho từng hyperparameter khi làm xong việc test fold = 3
  best_cv_df[sym].loc[best_cv_df[sym]['model']==model_name, 'best_hyper_param'] = top_5_hyperparameter
  best_cv_df[sym].loc[best_cv_df[sym]['model']==model_name, 'accuracy'] = acc
  return grid_search

# Hàm này lấy ra tất cả grid tốt nhaas và model mlp_clf đã thực hiện
def result_optimize_MLP(data_all,best_cv_df, symbols):
  grid_all = {}
  for sym in symbols["all"]:
    X_train = data_all[sym][0]
    y_train = data_all[sym][2]
    grid= optimize_model_MLP(X_train, y_train, best_cv_df, sym)
    grid_all[sym] = grid
  return grid_all


# Note cần lưu lại tập tốt nhất và lưu lại model tốt nhất

In [56]:
# import plotly.express as px
# for sym in symbols["all"]:
#   grid = grid_all[sym]
#   # model_name = 'mlp'

#   # best_cv_df[sym].loc[best_cv_df[sym]['model']==model_name, 'best_hyper_param'] = str(grid.best_params_)

#   cv_df = pd.DataFrame(grid.cv_results_)
#   # cv_values = cv_df.loc[grid.best_index_, cv_df.columns.str.startswith('split')].values
#   # best_cv_df[sym].loc[best_cv_df[sym]['model']==model_name, 'accuracy'] = cv_values[:5]

#   # thu được 5 model tốt nhất MLP
#   tune_result_df = pd.concat([pd.DataFrame(grid.cv_results_['params']), cv_df.loc[:,cv_df.columns.str.startswith('mean')] ], axis=1)
#   print(sym, "--------------------")

#   fig = px.parallel_coordinates(tune_result_df, color='mean_test_score')
#   fig.show()

# Kết hợp tất cả các model sau khi chạy kfold

In [57]:
from sklearn.neural_network import MLPClassifier
def best_model_stack(symbols, data_all, best_cv_df):
  estimators_all ={}
  res_df_all = {}
  for sym in symbols["all"]:
    X_train = data_all[sym][0]
    y_train = data_all[sym][2]
    X = np.array(X_train)
    y = np.array(y_train, dtype=np.int64)
    # DƯới đây là code vét cạn model
    for i_rf in range(0,1):
      for i_xgb in range(0,1):
        for i_mlp in range(0,1):
          # load 3 cấu hình tốt nhất cho 3 model vào biến rf xbg và mlp
          rf1 = RandomForestClassifier(**best_cv_df[sym].loc[best_cv_df[sym]['model']=='rf', 'best_hyper_param'].values[i_rf])
          xgb1 = XGBClassifier(**best_cv_df[sym].loc[best_cv_df[sym]['model']=='xgb', 'best_hyper_param'].values[i_xgb])
          mlp1 = MLPClassifier(**best_cv_df[sym].loc[best_cv_df[sym]['model']=='mlp', 'best_hyper_param'].values[i_mlp])
          # quá trình tổng hopkw 3 model để tạo ra model trung bình tốt nhất
          estimators = [rf1, xgb1, mlp1]
          estimators_name = 'rf_xgb_mlp'
          ensemble_name = 'average' + '_by_' + estimators_name + str(i_rf) + str(i_xgb) + str(i_mlp)
          res_df = CV_ensemble(ensemble_name, ensemble_average, estimators, X, y, n_folds=5, shuffle=True, random_state=config['random_state'])
          sym_brute_force = sym + str(i_rf) + str(i_xgb) + str(i_mlp)
          res_df_all[sym_brute_force] = res_df
          estimators_all[sym_brute_force] = estimators
          print(sym_brute_force)
          print(res_df)
    return res_df_all, estimators_all# Trả về những model vét cạn lại với nhau
# mô hình tốt nhất khi kết hợp 3 model khi chạy xong n_folds

# res_df_all,  estimators_all =  best_model_stack(symbols, data_all, best_cv_df)

In [58]:
def best_model_ens(res_df_all, symbols):
    keys = res_df_all.keys()
    best_model_all = {}  # Lấy model tốt nhất từ các model stack lại với nhau.
    for sym in symbols["all"]:
        model_max = 0
        symb = None
        for key in keys:  # Key chứa dữ liệu của các model stack lại với nhau và chưa luôn việc các cặp stack của model với nhau
            if sym in key:  # So sánh đúng là model thì ta tiếp tục
                candidate = res_df_all[key]['accuracy'].astype(float).mean()
                print(candidate)
                if candidate > model_max:
                    model_max = candidate
                    symb = key
        if symb is not None:
            best_model_all[sym] = {
                'model': res_df_all[symb]['model'].values[0],
                'accuracy': model_max  # Lưu giá trị accuracy
            }
    return best_model_all # Lấy ra cụ thể ở model nào mô hình nào tốt nhất vựa trên 3 con số cuối kết hợp với nhau
# best_model_all = best_model_ens(res_df_all, symbols)


In [59]:
def tuning_hyperparameter(data_all, symbols, best_cv_df):
  result_tune_and_grid = best_cv_rf_and_xgb(data_all, symbols, best_cv_df)
  grid_all = result_optimize_MLP(data_all, best_cv_df, symbols)
def check_tuning(symbols,best_cv_df):
  """Symbos chưa tất cả các mã của tiền tệ
  best_cv_df xem tập tuning của model để xem nó đã có hay chưa
  return trả về 1 cái list chứa sự tồn tại của từng mã
  """
  # Hàm này check xem symbols đó đã có hay chưa nếu chưa thì tuning cho model
  check_tun = {}
  for sym in symbols["all"]:
    if best_cv_df[sym] is not None:
      check_tun[sym] = True
    else:
      check_tun[sym] =  False
  return check_tun
def check_model(symbols, estimators_all):
  # Hàm này check xem model đó có tồn tại hay không
  check ={}
  for sym in symbols["all"]:
    # Check xem model đó đã tồn tại hãy chưa
    if estimators_all[sym] is not None:
      check[sym] = True
    else:
      check[sym] =  False
  return check
def train_model(sym, best_cv, X_train, y_train,best_index = "000"):
  """hÀM NÀy train model cân muốn train và có chỉ số cụ thể tốt nhất """
  i_rf = int(best_index[0])
  i_xgb = int(best_index[1])
  i_mlp = int(best_index[2])
  rf1 = RandomForestClassifier(**best_cv_df[sym].loc[best_cv_df[sym]['model']=='rf', 'best_hyper_param'].values[i_rf])
  xgb1 = XGBClassifier(**best_cv_df[sym].loc[best_cv_df[sym]['model']=='xgb', 'best_hyper_param'].values[i_xgb])
  mlp_clf = MLPClassifier()
  mlp1 = mlp_clf.set_params(**best_cv_df[sym].loc[best_cv_df[sym]['model']=='mlp', 'best_hyper_param'].values[i_mlp])



In [60]:
def predict_all(estimators_all, data_all, symbols):
    pred_y_label_all = {}
    for sym in symbols["all"]:
        if sym in data_all:
            estimators = estimators_all[sym + "000"]
            X_test = data_all[sym][1].copy()
            y_train = data_all[sym][2].copy()
            pred_y_label = ensemble_average_model(estimators, X_test, y_train)
            pred_y_label = pred_y_label.argmax(axis=1)
            pred_y_label_all[sym] = pred_y_label
        else:
            # Xử lý khi sym không tồn tại trong data_all
            print(f"KeyError: Symbol {sym} not found in data_all")
    return pred_y_label_all


In [61]:
# symbols = {"all": ["EURCHF"]}
# def exists_best_cv(file_path = "best_cv_df.pkl", data_all = data_all, symbols = symbols):
#   best_cv_df = delete_and_create_best(data_all, symbols)
#   if os.path.exists(file_path):
#     with open(file_path, 'rb') as file:
#       best_cv_df = pickle.load(file)
#     return best_cv_df, True
#   else:
#     return best_cv_df, False
# def check_exists_model():
#   pass
def predict_signal(symbols):
  data_forex = get_data(symbols)
  X,y = create_data(symbols, data_forex.copy())
  data_all = create_train_test_all_data_forex(X, y, symbols)
  # best_cv_df, check = exists_best_cv(file_path = "best_cv_df.pkl", data_all = data_all, symbols = symbols)
  best_cv_df = delete_and_create_best(data_all, symbols)
  print(best_cv_df)
  tuning_hyperparameter(data_all, symbols, best_cv_df)
  with open("best_cv_df.pkl", 'wb') as file:
    pickle.dump(best_cv_df, file)
  res_df_all, estimators_all=  best_model_stack(symbols, data_all, best_cv_df)
  best_model_all = best_model_ens(res_df_all, symbols)
  with open("best_model_all.pkl", 'wb') as file:
        pickle.dump(best_model_all, file)
  predict = predict_all(estimators_all, data_all, symbols)
  return predict, best_model_all
predict, best_model = predict_signal(symbols)

USDCHF
USDCHF
{'USDCHF':    model accuracy best_hyper_param
0     rf     None             None
1     rf     None             None
2     rf     None             None
3     rf     None             None
4     rf     None             None
5    xgb     None             None
6    xgb     None             None
7    xgb     None             None
8    xgb     None             None
9    xgb     None             None
10   mlp     None             None
11   mlp     None             None
12   mlp     None             None
13   mlp     None             None
14   mlp     None             None}
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 8 candidates, totalling 24 fits


Eval_CV:   0%|          | 0/5 [00:00<?, ?it/s]

USDCHF000
                      model              log_loss            accuracy
0  average_by_rf_xgb_mlp000   -0.4779648190944229  0.7894736842105263
1  average_by_rf_xgb_mlp000  -0.44412003948451423  0.8170426065162907
2  average_by_rf_xgb_mlp000   -0.4583936731049038  0.8170426065162907
3  average_by_rf_xgb_mlp000   -0.4241795817116988  0.8467336683417085
4  average_by_rf_xgb_mlp000   -0.4472632148729563   0.821608040201005
0.8183801211571643


In [62]:
print(predict)

{'USDCHF': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)}


In [63]:
best_model

{'USDCHF': {'model': 'average_by_rf_xgb_mlp000',
  'accuracy': 0.8183801211571643}}

In [1]:
!pip show pandas

Name: pandas
Version: 1.5.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: C:\Users\Admin\anaconda3\Lib\site-packages
Requires: numpy, numpy, python-dateutil, pytz
Required-by: altair, bokeh, catboost, datashader, finta, holoviews, hvplot, medmnist, mizani, panel, plotnine, pmdarima, seaborn, statsmodels, streamlit, ta, xarray


In [3]:
import pickle

# Open the file in binary read mode
with open('D:/ForexResearch/best_cv_df.pkl', 'rb') as file:
    data = pickle.load(file)

# Now 'data' contains the deserialized object from the .pkl file
print(data)

{'FPT':    model accuracy                                   best_hyper_param
0     rf  0.67119  {'criterion': 'entropy', 'max_depth': 8, 'n_es...
1     rf  0.67107  {'criterion': 'entropy', 'max_depth': 10, 'n_e...
2     rf  0.67108  {'criterion': 'entropy', 'max_depth': 8, 'n_es...
3     rf  0.67042  {'criterion': 'entropy', 'max_depth': 10, 'n_e...
4     rf  0.67012  {'criterion': 'entropy', 'max_depth': 10, 'n_e...
5    xgb  0.63638  {'colsample_bytree': 1.0, 'gamma': 0.5, 'learn...
6    xgb  0.63687  {'colsample_bytree': 1.0, 'gamma': 0.5, 'learn...
7    xgb  0.65213  {'colsample_bytree': 1.0, 'gamma': 0.5, 'learn...
8    xgb  0.64781  {'colsample_bytree': 1.0, 'gamma': 0.5, 'learn...
9    xgb      NaN  {'colsample_bytree': 2, 'gamma': 0.5, 'learnin...
10   mlp  0.66871  {'activation': 'tanh', 'alpha': 0.05, 'hidden_...
11   mlp  0.66113  {'activation': 'tanh', 'alpha': 0.1, 'hidden_l...
12   mlp  0.66289  {'activation': 'tanh', 'alpha': 0.05, 'hidden_...
13   mlp  0.66404  {'activ